# Train Random Forest with SKLearn

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier as sklRF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import airline_data_local

In [ ]:
print('pandas version:', pd.__version__)
print('SKLearn version:', sklearn.__version__)

In [ ]:
data_dir = '/datasets/data_airline'

In [ ]:
# from airline_data_local import prepare_airline_data
# prepare_airline_data(data_dir, 20000000)

In [ ]:
%%time
df = pd.read_parquet(os.path.join(data_dir, 'airline_20m.parquet'))

## Train model

In [ ]:
# Encode categoricals as numeric
for col in df.select_dtypes(['object']).columns:
    df[col] = df[col].astype('category').cat.codes.astype(np.float32)

# cast all remaining columns to float32
for col in df.columns:
    if col in df.select_dtypes(['object']).columns: pass
    else:
        df[col] = df[col].astype(np.float32)
    
df['ArrDelayBinary'] = 1 * (df['ArrDelay'] > 0)
    
X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
y = df['ArrDelayBinary'].astype(np.int32)
del df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77, test_size=0.2)

In [ ]:
%%time
skl_rf = sklRF(n_estimators=100, max_depth=16, max_features=4, n_jobs=8)
skl_rf.fit(X_train, y_train)

In [ ]:
skl_pred = skl_rf.predict(X_test)

In [ ]:
skl_accuracy = accuracy_score(skl_pred, y_test) * 100
print('Accuracy: sklRF {:.2f}%'.format(skl_accuracy))